In [1]:
!pip install vit-pytorch
!pip install albumentations==0.4.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 4.2 MB/s 
     |████████████████████████████████| 948 kB 29.0 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=c0be8419528ed9140907d6bce55398ca39a6771ee54ab14a4894ccaee1617ebc
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumen

In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/AI대회')

In [3]:
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import glob
import numpy
import random

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve

import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
import torchvision.models as models
from sklearn.model_selection import train_test_split
cudnn.benchmark = True


In [4]:
data = pd.read_csv('train.csv')
hz_columns = list(data.columns[1:])

In [5]:
data['leaktype'].unique()

array(['out', 'in', 'normal', 'other', 'noise'], dtype=object)

In [6]:
class_mapping = {'normal':0, 'out':1, 'in':2, 'noise':3, 'other':4}
class_mapping_inv = {value:key for key,value in class_mapping.items()}

In [7]:
class_mapping_inv

{0: 'normal', 1: 'out', 2: 'in', 3: 'noise', 4: 'other'}

In [8]:
data['leaktype_num'] = data['leaktype'].map(lambda x:class_mapping[x])

In [9]:
data['leaktype_onehot'] = data['leaktype_num'].map(lambda x:np.eye(5)[x])

In [10]:
data['leaktype_onehot']

0        [0.0, 1.0, 0.0, 0.0, 0.0]
1        [0.0, 1.0, 0.0, 0.0, 0.0]
2        [0.0, 1.0, 0.0, 0.0, 0.0]
3        [0.0, 1.0, 0.0, 0.0, 0.0]
4        [0.0, 1.0, 0.0, 0.0, 0.0]
                   ...            
33595    [0.0, 0.0, 0.0, 1.0, 0.0]
33596    [0.0, 0.0, 0.0, 1.0, 0.0]
33597    [0.0, 0.0, 0.0, 1.0, 0.0]
33598    [0.0, 0.0, 0.0, 1.0, 0.0]
33599    [0.0, 0.0, 0.0, 1.0, 0.0]
Name: leaktype_onehot, Length: 33600, dtype: object

In [11]:
traindata, valdata = train_test_split(data, test_size=0.2, random_state=100)

In [12]:
traindata = traindata.reset_index(drop=True)
valdata = valdata.reset_index(drop=True)

In [59]:
class LeakDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def shuffle(self):
        self.dataframe = sklearn.utils.shuffle(self.dataframe).reset_index(drop=True)
        
    def __getitem__(self, idx):
        # leaktype = self.dataframe.loc[idx, 'leaktype_num']
        leaktype_onehot = self.dataframe.loc[idx, 'leaktype_onehot']
        before_image = np.array(self.dataframe.loc[idx, hz_columns])
        # print(before_image.shape)

        # image_mean = before_image.mean()
        # before_image_long = np.concatenate([before_image, np.repeat(image_mean, 16)])
        converted_image = np.reshape(before_image, (27, 19))

        converted_image = converted_image.astype(np.uint8)
        converted_image = cv2.cvtColor(converted_image, cv2.COLOR_GRAY2RGB)
        # print(converted_image.shape)
        if self.transform is not None:
            converted_image = self.transform(image=converted_image)["image"].float()
        
        #이미지와 온핫 라벨들을 반환한다.
        return converted_image, leaktype_onehot

class LeakTestDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        before_image = np.array(self.dataframe.loc[idx, hz_columns])
        # print(before_image.shape)

        # image_mean = before_image.mean()
        # before_image_long = np.concatenate([before_image, np.repeat(image_mean, 16)])
        converted_image = np.reshape(before_image, (27, 19))

        converted_image = converted_image.astype(np.uint8)
        converted_image = cv2.cvtColor(converted_image, cv2.COLOR_GRAY2RGB)
        # print(converted_image.shape)
        if self.transform is not None:
            converted_image = self.transform(image=converted_image)["image"].float()
        
        #이미지와 온핫 라벨들을 반환한다.
        return converted_image

In [30]:
EPOCHS = 200
BATCH_SIZE = 16
LEARNING_RATE = 0.003
train_transforms = A.Compose(
    [
        # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.Resize(32, 32),
        ToTensorV2(),
    ]
)
test_transforms = A.Compose(
    [
        # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.Resize(32, 32),
        ToTensorV2(),
    ]
)

In [31]:
train_dataset = LeakDataset(traindata, train_transforms)
test_dataset = LeakDataset(valdata, train_transforms)

In [32]:
len(train_dataset), len(test_dataset)

(26880, 6720)

In [17]:
class CustomEffNet(torch.nn.Module):
    def __init__(self):
        super(CustomEffNet, self).__init__()
        
        effnet = getattr(models, params["model"])(pretrained=True, )
        modules = list(effnet.children())[:-1]
        self.effnet = nn.Sequential(*modules)
        self.linear_label = torch.nn.Linear(1280, 5)
      
    def forward(self, images):
        feature = self.effnet(images)
        # print(feature.shape)
        feature = feature.view(-1, 1280)
        label = self.linear_label(feature)
        
        return label

In [18]:
def calculate_accuracy(output, target):
    output_arg = torch.argmax(output, dim=1)
    target_arg = torch.argmax(target, dim=1)
    return torch.true_divide((output_arg == target_arg).sum(dim=0), output.size(0)).item()

In [19]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"], float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

In [20]:
params = {
    "model": "efficientnet_b0",
    "device": "cuda:0",
    "lr": 0.001,
    "batch_size": 64,
    "num_workers": 16,
    "epochs": 300,
}

In [21]:
model = CustomEffNet().to(params["device"])

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [22]:
criterion = nn.CrossEntropyLoss().to(params["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

In [33]:
train_loader = DataLoader(
    train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=params["num_workers"], pin_memory=True,
)
val_loader = DataLoader(
    test_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [92]:
def train(train_loader, model, criterion, optimizer, epoch, params):
    metric_monitor = MetricMonitor()
    # model 학습용으로 전환
    model.train()
    stream = tqdm(train_loader)
    for i, (images, target_onehot) in enumerate(stream, start=1):
        images = images.to(params["device"], non_blocking=True)
        # 데이터 로딩 후 target은 4가지로 나뉨 
        # target = target.to(params["device"], non_blocking=True).float().view(images.shape[0], -1)
        target_onehot = target_onehot.to(params["device"], non_blocking=True).float().view(images.shape[0], -1)
        
        # model에 이미지를 입력해서 출력을 받는다.
        output = model(images)
        
        # 출력과 라벨의 loss를 계산한다.
        loss = criterion(output, target_onehot)
        
        # 출력과 라벨의 정확도를 계산한다.
        accuracy = calculate_accuracy(output, target_onehot)
        
        # metric_monitor에 각 loss를 업데이트 한다.
        metric_monitor.update("loss", loss.item())
        
        # metric_monitor에 각 정확도를 업데이트 한다.
        metric_monitor.update("accuracy", accuracy)
        
        # 옵티마이저 함수의 gradient를 0로 다시 설정한다.
        optimizer.zero_grad()
        
        #loss에 backward를 주어 backpropagation을 통한 학습을 진행하게 한다.
        loss.backward(retain_graph=True)
        
        # 옵티마이저에서 한 스텝 진행한다.
        optimizer.step()
        # 해당 에포크에 학습 관련 로그를 남긴다.
        stream.set_description(
            "Ep:{epoch}. Train {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor))
    # 평균 loss를 리턴한다.
    return loss

In [93]:
def validate(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    # 모델을 평가용으로 전환한다.
    model.eval()
    stream = tqdm(val_loader)
    # gpu 메모리 loss를 방지하기 위해 torch.no_grad를 써준다.
    with torch.no_grad():
        for i, (images, target_onehot) in enumerate(stream, start=1):
            images = images.to(params["device"], non_blocking=True)
            # 이미지와 라벨을 불러온다.
            # target = target.to(params["device"], non_blocking=True).float().view(images.shape[0], -1)
            target_onehot = target_onehot.to(params["device"], non_blocking=True).float().view(images.shape[0], -1)
            
            # 이미지를 모델에 입력하여 출력을 저장한다.
            output = model(images)

            # 출력물과 라벨 사이의 loss를 계산한다.
            loss = criterion(output, target_onehot)

            # 출력물과 라벨 사이의 정확도를 계산한다.
            accuracy = calculate_accuracy(output, target_onehot)
          
            # metric_monitor에 loss를 업데이트 한다.
            metric_monitor.update("loss", loss.item())

            # metric_monitor에 정확도를 업데이트 한다.
            metric_monitor.update("accuracy", accuracy)
            
            # 해당 에포크에 평가 관련 로그를 남긴다.
            stream.set_description(
                "Ep: {epoch}. Val. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor))
    # 평균 loss를 리턴한다.
    return loss

In [36]:
# load_model = torch.load('model_best_220615_3232.pt')

In [37]:
# model.load_state_dict(load_model['model_state_dict'])

In [40]:
# check version number
import imblearn
from imblearn.over_sampling import RandomOverSampler 
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [41]:
oversample = SMOTE()

In [62]:
data

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ,leaktype_num,leaktype_onehot
0,out,0,2,2,0,2,0,2,2,2,...,2,2,5,2,2,5,2,5,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
1,out,0,0,0,3,0,3,0,0,0,...,3,3,3,3,3,3,3,6,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
2,out,0,4,4,4,4,5,4,4,5,...,6,5,6,6,6,5,6,4,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
3,out,0,6,5,5,6,5,6,6,5,...,7,7,5,6,5,5,7,7,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
4,out,0,3,0,0,3,0,0,3,3,...,3,3,3,3,3,3,3,3,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33595,noise,0,5,6,6,23,15,7,10,15,...,7,7,7,7,6,7,6,7,3,"[0.0, 0.0, 0.0, 1.0, 0.0]"
33596,noise,0,3,3,3,2,3,3,3,3,...,5,5,5,3,3,3,3,3,3,"[0.0, 0.0, 0.0, 1.0, 0.0]"
33597,noise,0,0,0,0,0,0,0,0,0,...,0,0,0,3,3,0,0,3,3,"[0.0, 0.0, 0.0, 1.0, 0.0]"
33598,noise,0,8,7,7,5,8,8,8,8,...,8,8,10,10,7,7,7,5,3,"[0.0, 0.0, 0.0, 1.0, 0.0]"


In [76]:
X_smote, Y_smote = oversample.fit_resample(data[hz_columns], data['leaktype_num'])

In [79]:
Y_smote = pd.DataFrame(Y_smote)

In [80]:
Y_smote['leaktype_onehot'] = Y_smote['leaktype_num'].map(lambda x:np.eye(5)[x])

In [83]:
new_data = pd.concat([X_smote, Y_smote], axis=1)

In [84]:
new_data

,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,90HZ,...,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ,leaktype_num,leaktype_onehot
0,0,2,2,0,2,0,2,2,2,2,...,2,2,5,2,2,5,2,5,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
1,0,0,0,3,0,3,0,0,0,0,...,3,3,3,3,3,3,3,6,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
2,0,4,4,4,4,5,4,4,5,5,...,6,5,6,6,6,5,6,4,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
3,0,6,5,5,6,5,6,6,5,5,...,7,7,5,6,5,5,7,7,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
4,0,3,0,0,3,0,0,3,3,3,...,3,3,3,3,3,3,3,3,1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,0,3,2,2,3,3,2,4,3,4,...,5,5,5,5,5,5,4,5,4,"[0.0, 0.0, 0.0, 0.0, 1.0]"
89996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,"[0.0, 0.0, 0.0, 0.0, 1.0]"
89997,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,4,"[0.0, 0.0, 0.0, 0.0, 1.0]"
89998,0,41,40,44,36,38,37,38,35,36,...,4,4,5,4,4,4,5,4,4,"[0.0, 0.0, 0.0, 0.0, 1.0]"


In [ ]:

model = model.to(params['device'])

for k, (train_index, test_index) in enumerate(kf.split(new_data)):
    minloss = 1e7
    traindata = new_data.loc[train_index].reset_index(drop=True)
    valdata = new_data.loc[test_index].reset_index(drop=True)

    train_dataset = LeakDataset(traindata, train_transforms)
    test_dataset = LeakDataset(valdata, train_transforms)

    train_loader = DataLoader(
    train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=params["num_workers"], pin_memory=True,
    )
    val_loader = DataLoader(
        test_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
    )
    for epoch in range(1, params["epochs"] + 1):
        # 학습을 진행한다.
        trainloss = train(train_loader, model, criterion, optimizer, epoch, params)
        # 평가를 진행한다.
        thisloss = validate(val_loader, model, criterion, epoch, params)
        
        if thisloss < minloss:
            # 평가 loss가 최솟값일 경우 모델을 저장한다.
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': trainloss.item(),
                'val_loss' : minloss
                }, f'model_best_220617_3232_{k}fold.pt')
            minloss = thisloss

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Ep:1. Train loss: 3.051 | accuracy: 0.543:   6%|▌         | 64/1125 [00:10<01:48,  9.78it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f68e489ee60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f68e489ee60>
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_w

In [30]:
test_csv = pd.read_csv('test.csv')

In [31]:
test_csv

,id,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
0,A8omWrwmLF,0,4,4,4,5,4,4,5,4,...,6,5,5,7,5,5,5,5,5,6
1,XEVCsLi6ps,0,8,7,10,10,10,10,10,10,...,14,14,11,13,11,13,13,11,11,11
2,cEVCc0xZkC,0,0,0,0,3,0,0,0,0,...,3,0,0,3,3,0,0,3,0,0
3,M2a2qkpvQ4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
4,Sts4cwqBEo,0,4,4,5,5,6,9,5,7,...,7,8,6,5,7,5,7,7,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7815,XPZiUyQPgg,0,3,4,3,4,3,4,4,3,...,5,5,5,5,7,5,5,6,5,5
7816,fLSro59Y12,0,10,12,13,20,42,37,26,26,...,10,10,7,10,10,9,9,9,10,10
7817,C3guvrwoiN,0,4,4,4,4,4,4,5,4,...,7,7,7,6,7,6,6,5,6,6
7818,aU6pSfYjZG,0,8,7,10,10,9,7,9,9,...,10,13,16,15,15,17,18,17,11,13


In [32]:
sample = pd.read_csv('sample_submission.csv')

In [33]:
realtest_dataset = LeakTestDataset(test_csv, train_transforms)

In [34]:
test_loader = DataLoader(
    realtest_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [35]:
model.eval()
pred_args = []
with torch.no_grad():
    for testdata in test_loader:
        pred = model(testdata.to('cuda:0'))
        pred_arg = torch.argmax(pred, dim=1).detach().cpu().numpy()
        pred_args.append(pred_arg)
pred_args = np.concatenate(pred_args)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [36]:
sorter = list(sample['id'])
test_ids = test_csv['id']
y_pred_df = pd.DataFrame(pred_args, columns=['leaktype'])
y_pred_df['leaktype'] = y_pred_df['leaktype'].replace(class_mapping_inv)
pred_df = pd.concat([test_ids, y_pred_df],axis=1)

In [37]:
# sort predictions
resdf = pred_df.set_index('id')
result = resdf.loc[sorter].reset_index()

In [38]:
result.to_csv('result_v3.csv', index=False)

In [40]:
from sklearn.metrics import f1_score

In [41]:
def cal_f1_score(model, data_loader):
    model.eval()
    pred_args = []
    true_args = []
    with torch.no_grad():
        for testdata in data_loader:
            images, targets, targets_onehot = testdata
            pred = model(images.to('cuda:0'))
            pred_arg = torch.argmax(pred, dim=1).detach().cpu().numpy()
            pred_args.append(pred_arg)
            true_args.append(targets)
    true_args = np.concatenate(true_args)
    pred_args = np.concatenate(pred_args)

    print(f"f1 score is ... {f1_score(true_args, pred_args, average='macro')}")
    return true_args, pred_args

In [42]:
true_args, pred_args = cal_f1_score(model, val_loader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


f1 score is ... 0.8164739872591833


In [43]:
true_args, pred_args = cal_f1_score(model, train_loader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


f1 score is ... 0.9877459574314598


In [ ]:
model = LGBMClassifier(**paramenter)